In [2]:
%matplotlib widget
import pandas as pd
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os.path
import sys
import numpy as np


In [3]:
# Define set of all parameters and meta-parameters
    parameters = {    
    "delta_schooling":[0.5],
        "xi":[0, 30 * 37199.03, 140 * 37199.03],
#         , 140 * 37199.03],
        "icus":[3000],
        "tests":[0],
        "frequencies":[(7,14)],
        "eta":[0,0.1]
    }
    
    meta_parameters = {
        "trust_region_radius":[0,0.05,0.1,0.2,0.4,0.6],
        "max_inner_iterations_mult":[1, 1.5, 2],
        "initial_uhat":["dynamic_gradient", "full_lockdown", "full_open"]
    }
    
    n_days = 90
    groups = "all"


In [4]:
# Add all data into one big pandas dataframe

ls = []

for delta in parameters["delta_schooling"]:
    for xi in parameters["xi"]:
        for icus in parameters["icus"]:
            for tests in parameters["tests"]:
                for freq in parameters["frequencies"]:
                    for eta in parameters["eta"]:
                        for initial_uhat in meta_parameters["initial_uhat"]:
                            with open(f"testing_outputs_ndays={n_days}_eta={eta}_tests={tests}_xi={xi*10}_freq={freq[1]}_groups={groups}_initial_uhat={initial_uhat}_opt_bouncing=False.csv", "r") as file:
                                df = pd.read_csv(file, header=None, names=["radius", "factor", "num_iterations", "reward", "time"])
                            if len(df) == 0:
                                print(file.name)
                            df["delta"] = delta
                            df["xi"] = xi
                            df["icus"] = icus
                            df["tests"] = tests
                            df["freq"] = freq[1]
                            df["eta"] = eta
                            df["initial_uhat"] = initial_uhat
                            ls.append(df)

all_data = pd.concat(ls, ignore_index=True)


In [5]:
# First, look for each combination of parameters what combination of meta-param achieves the best rewards

for delta in parameters["delta_schooling"]:
    for xi in parameters["xi"]:
        for icus in parameters["icus"]:
            for tests in parameters["tests"]:
                for freq in parameters["frequencies"]:
                    for eta in parameters["eta"]:
                        maximum_reward = all_data[(all_data.delta == delta) 
                                                 & (all_data.xi == xi) 
                                                 & (all_data.icus == icus)
                                                 & (all_data.tests == tests) 
                                                 & (all_data.freq == freq[1])
                                                 & (all_data.eta == eta)].reward.max()
                        print(f"For xi={xi} and eta={eta}, the maximum reward is: {maximum_reward}")
                        print(f"And these are the meta-parameters that achieve it:")
                        print("Observe that radius == 0 implies that the solution follows the initial uhat")
                        print(all_data[all_data.reward == maximum_reward])
                        print("")

For xi=0 and eta=0, the maximum reward is: 55655867.97208577
And these are the meta-parameters that achieve it:
Observe that radius == 0 implies that the solution follows the initial uhat
    radius  factor  num_iterations        reward         time  delta   xi  \
3     0.05     1.0       20.000000  5.565587e+07  1385.065299    0.5  0.0   
4     0.05     1.5       30.000000  5.565587e+07  1395.110099    0.5  0.0   
5     0.05     2.0       40.000000  5.565587e+07  1408.321419    0.5  0.0   
6     0.10     1.0       10.000000  5.565587e+07  1170.008036    0.5  0.0   
7     0.10     2.0       20.000000  5.565587e+07  1187.123408    0.5  0.0   
8     0.10     1.5       15.000000  5.565587e+07  1190.768425    0.5  0.0   
9     0.20     1.0        5.000000  5.565587e+07  1060.251859    0.5  0.0   
10    0.20     2.0       10.000000  5.565587e+07  1089.156359    0.5  0.0   
11    0.20     1.5        7.500000  5.565587e+07  1089.570733    0.5  0.0   
12    0.60     1.0        1.666667  5.5655

Notice above that for any combination of parameters we achieve the maximum reward for some combination of meta_param

In [6]:
# Now we'll define the ratio of the rewards wrt to the maximum reward 

for delta in parameters["delta_schooling"]:
    for xi in parameters["xi"]:
        for icus in parameters["icus"]:
            for tests in parameters["tests"]:
                for freq in parameters["frequencies"]:
                    for eta in parameters["eta"]:
                        maximum_reward = all_data[(all_data.delta == delta) 
                                                 & (all_data.xi == xi) 
                                                 & (all_data.icus == icus)
                                                 & (all_data.tests == tests) 
                                                 & (all_data.freq == freq[1])
                                                 & (all_data.eta == eta)].reward.max()
                        
                        all_data.loc[(all_data.delta == delta)
                                                 & (all_data.xi == xi) 
                                                 & (all_data.icus == icus)
                                                 & (all_data.tests == tests) 
                                                 & (all_data.freq == freq[1])
                                                 & (all_data.eta == eta), "FractionalReward"] = all_data[(all_data.delta == delta) 
                                                 & (all_data.xi == xi) 
                                                 & (all_data.icus == icus)
                                                 & (all_data.tests == tests) 
                                                 & (all_data.freq == freq[1])
                                                 & (all_data.eta == eta)].reward / maximum_reward


In [7]:
# Now for each combination of meta-parameters we can plot a histogram of the fractional rewards.
# This should give an idea of how good the meta-parameters are for all the instances.


# Here you can choose which combination of meta-param to plot
meta_parameters = {
        "trust_region_radius":[0,0.05,0.1,0.2,0.4,0.6],
        "max_inner_iterations_mult":[1, 1.5, 2],
        "initial_uhat":["dynamic_gradient", "full_lockdown", "full_open"]
    }


#You can choose which combination of parameters to plot
xis = [0, 30 * 37199.03, 140 * 37199.03]
# , 140 * 37199.03]
etas = [0,0.1]


times = []
frac_rewards = []
param = []

for radius in meta_parameters["trust_region_radius"]:
    for factor in meta_parameters["max_inner_iterations_mult"]:
        
        frac_reward_list_rad_fact = []
        running_times = []
        for initial_uhat in meta_parameters["initial_uhat"]:
#             print(f"Meta Param: radius = {radius}, factor = {factor}, num iterations = {factor/radius if radius>0 else 0}, initia uhat = {initial_uhat}")
            frac_rewards_list = list(all_data.loc[(all_data.radius==radius) &
                                                  (all_data.factor==factor) & 
                                                  (all_data.initial_uhat==initial_uhat) &
                                                  (all_data.xi.isin(xis)) &
                                                  (all_data.eta.isin(etas)), "FractionalReward"])
            partial_running_times = list(all_data.loc[(all_data.radius==radius) &
                                                  (all_data.factor==factor) & 
                                                  (all_data.initial_uhat==initial_uhat) &
                                                  (all_data.xi.isin(xis)) &
                                                  (all_data.eta.isin(etas)), "time"])
#             print(f"List of Fractional Rewards: {frac_rewards_list}")
            frac_reward_list_rad_fact += frac_rewards_list
            running_times += partial_running_times
#             print(f"Average Fractional Reward: {np.mean(frac_rewards_list)}")
            num_bins = 128
#             print("Histogram of Fractional Rewards:")
#             plt.hist(frac_rewards_list, num_bins, facecolor='blue', alpha=0.5)
#             plt.show()
        print("-----------------------------------------------------")
        print(f"Meta Param: radius = {radius}, factor = {factor}, num iterations = {factor/radius if radius>0 else 0}")
        print(f"Average Fractional Reward across starting points: {np.mean(frac_reward_list_rad_fact)}")
        print(f"Average time across starting points: {np.mean(running_times)}")
        times.append(np.mean(running_times))
        frac_rewards.append(np.mean(frac_reward_list_rad_fact))
        param.append((radius, factor))

# plt.scatter(times[3:], frac_rewards[3:])
# plt.show()

fig, ax = plt.subplots()
ax.scatter(times[3:], frac_rewards[3:])

for i, txt in enumerate(param[3:]):
    ax.annotate(txt, (times[i], frac_rewards[i]))

-----------------------------------------------------
Meta Param: radius = 0, factor = 1, num iterations = 0
Average Fractional Reward across starting points: -3.408234242719686
Average time across starting points: 948.3297182718912
-----------------------------------------------------
Meta Param: radius = 0, factor = 1.5, num iterations = 0
Average Fractional Reward across starting points: -3.408234242719686
Average time across starting points: 950.0732090870539
-----------------------------------------------------
Meta Param: radius = 0, factor = 2, num iterations = 0
Average Fractional Reward across starting points: -3.408234242719686
Average time across starting points: 949.4243554936515
-----------------------------------------------------
Meta Param: radius = 0.05, factor = 1, num iterations = 20.0
Average Fractional Reward across starting points: 0.9903751516101733
Average time across starting points: 3466.9894530110887
-----------------------------------------------------
Meta 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…